In [1]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
import optuna

import os
import sys
import math
import itertools
import numpy as np
import pandas as pd
from scipy.special import perm, comb

from google.cloud import bigquery

In [3]:
client = bigquery.Client()

In [4]:
sql = """
select
  race_info_id,
  race_date,
  place,
  race_num,
  umakazu
from
  [dataset].[table]
where
  race_date < date('2019-06-21')
group by
  race_info_id,
  race_date,
  place,
  race_num,
  umakazu
order by
  race_info_id
"""

In [5]:
df = client.query(sql).to_dataframe()
df.head()

,race_info_id,race_date,place,race_num,umakazu
0,192991,2019-01-01,川崎,1,7
1,192992,2019-01-01,川崎,2,10
2,192993,2019-01-01,川崎,3,11
3,192994,2019-01-01,川崎,4,10
4,192995,2019-01-01,川崎,5,10


In [6]:
results = client.query("""
select
  r.*  
from (
  select distinct race_info_id from
    [dataset].[table_run]
  where
    race_date < date('2019-06-21')
) as x
  inner join
    [dataset].[table_result] as r
    on r.race_info_id = x.race_info_id
order by
  r.race_info_id
""").to_dataframe()

In [7]:
scores = client.query("""
select
  x.race_info_id,
  x.uma_id,
  x.umaban,
  IF (x.umakazu < y.max_umaban, y.max_umaban, x.umakazu) as umakazu,
  r.score,
  r.value
from (
  select
      race_info_id,
      uma_id,
      umaban,
      umakazu
  from
    [dataset].[table_run] as kr
  where
    race_date < date('2019-06-21')
) as x
  inner join (
    select
      cast(uma_id as INT64) as uma_id,
      cast(race_info_id as INT64) as race_info_id,
      avg(predicted.tables.score) as score,
      predicted.tables.value as value
    from
      [predicted_dataset].predictions as p
      CROSS JOIN UNNEST(p.predicted_is_profit) as predicted
    where
      predicted.tables.value = '1'
    group by
      p.uma_id,
      p.race_info_id,
      predicted.tables.value
  ) as r
    on r.race_info_id = x.race_info_id
       and r.uma_id = x.uma_id
  inner join (
    select
      race_info_id,
      max(umaban) as max_umaban
    from
      [dataset].[table_run]
    group by
      race_info_id
  ) as y
    on y.race_info_id = x.race_info_id
order by
  r.race_info_id
""").to_dataframe()
scores.head()

,race_info_id,uma_id,umaban,umakazu,score,value
0,192991,14036,2,8,0.529314,1
1,192991,13703,3,8,0.306875,1
2,192991,12891,4,8,0.569858,1
3,192991,13319,5,8,0.370065,1
4,192991,59473,6,8,0.402599,1


In [8]:
def umaban2waku(umakazu):
    m = 8

    a = m * 2 - umakazu
    n = 0
    step = 1
    ws = {}
    for i in range(umakazu):
        if i >= a:
            if step == 0:
                step = 1
            else:
                n += 1
                step = 0
        else:
            n += 1
        ws[i+1] = n
    return ws

In [9]:
def tanpuku(score_df, select_num):
    return [[int(x)] for x in list(score_df[:select_num].kumiban)]

In [10]:
def uma_ren(score_df, sub, select_num, ticket_unit):
    kumibans = list(score_df.kumiban)

    pairs = []
    if sub == 'box':
        pairs = list(itertools.combinations(kumibans[:select_num], ticket_unit))
    elif sub == 'flow':
        jiku = kumibans[0]
        pairs = [ (jiku, x) for x in kumibans[1:] ]
    elif sub == 'form2':
        jiku1 = kumibans[0]
        jiku2 = kumibans[1]

        pairs.append((jiku1, jiku2))
        pairs.append((jiku1, jiku2))
        pairs += [ (jiku1, x) for x in kumibans[2:] ]
        pairs += [ (jiku2, x) for x in kumibans[2:] ]

    pairs = [sorted(x) for x in pairs ]
    return pairs

In [11]:
def uma_tan(score_df, sub, select_num, ticket_unit):
    kumibans = list(score_df.kumiban)

    pairs = []
    if sub == 'mbox':
        pairs = list(itertools.permutations(kumibans[:select_num], ticket_unit))
    elif sub == 'flow':
        jiku = kumibans[0]
        pairs = [ (jiku, x) for x in kumibans[1:] ]
    elif sub == 'mflow':
        jiku = kumibans[0]
        pairs += [ (jiku, x) for x in kumibans[1:] ]
        pairs += [ (x, jiku) for x in kumibans[1:] ]

    return pairs

In [12]:
def sanren_puku(score_df, sub, select_num, ticket_unit):
    kumibans = list(score_df.kumiban)

    pairs = []
    if sub == 'box':
        pairs = list(itertools.combinations(kumibans[:select_num], ticket_unit))
    elif sub == 'flow1':
        jiku = kumibans[0]
        others = list(itertools.combinations(kumibans[1:select_num], 2))
        pairs = [ (jiku, x[0], x[1]) for x in others ]
    elif sub == 'flow2':
        jiku1 = kumibans[0]
        jiku2 = kumibans[1]
        pairs += [ (jiku1, jiku2, x) for x in kumibans[2:] ]
    elif sub == 'form21':
        jiku1 = kumibans[0]
        jiku2 = kumibans[1]
        others1 = list(itertools.combinations(kumibans[1:select_num], 2))
        others2 = list(itertools.combinations(kumibans[2:select_num], 2))
        pairs += [ (jiku1, x[0], x[1]) for x in others1 ]
        pairs += [ (jiku2, x[0], x[1]) for x in others2 ]
    elif sub == 'form22':
        jiku1 = kumibans[0]
        jiku2 = kumibans[1]
        jiku3 = kumibans[2]
        pairs += [ (jiku1, jiku2, x) for x in kumibans[2:] ]
        pairs += [ (jiku2, jiku3, x) for x in kumibans[3:] ]  # jiku1とjiku3 からの流しが無い...！

    pairs = [sorted(x) for x in pairs ]
    return pairs

In [13]:
def sanren_tan(score_df, sub, select_num, ticket_unit):
    kumibans = list(score_df.kumiban)

    pairs = []
    if sub == 'mbox':
        pairs = list(itertools.permutations(kumibans[:select_num], ticket_unit))
    elif sub == 'tflow1':
        jiku = kumibans[0]
        others = list(itertools.permutations(kumibans[1:select_num], 2))
        pairs = [ (jiku, x[0], x[1]) for x in others ]
    elif sub == 'tflow2':
        jiku1 = kumibans[0]
        jiku2 = kumibans[1]
        pairs = [ (jiku1, jiku2, x) for x in kumibans[2:] ]
    elif sub == 'mflow1':
        jiku = kumibans[0]
        for i in range(len(kumibans) - 2):
            pairs += list(itertools.permutations([jiku, kumibans[i+1], kumibans[i+2]]))
    elif sub == 'mflow2':
        jiku1 = kumibans[0]
        jiku2 = kumibans[1]
        for i in range(len(kumibans) - 2):
            pairs += list(itertools.permutations([jiku1, jiku2, kumibans[i+2]]))
    elif sub == 'form2t1':
        jiku1 = kumibans[0]
        jiku2 = kumibans[1]
        others1 = list(itertools.permutations(kumibans[1:select_num], 2))
        others2 = list(itertools.permutations(kumibans[2:select_num], 2))
        pairs += [ (jiku1, x[0], x[1]) for x in others1 ]
        pairs += [ (jiku2, x[0], x[1]) for x in others2 ]
    elif sub == 'form2t2':
        jiku1 = kumibans[0]
        jiku2 = kumibans[1]
        jiku3 = kumibans[2]
        pairs += [ (jiku1, jiku2, x) for x in kumibans[2:] ]
        pairs += [ (jiku2, jiku3, x) for x in kumibans[3:] ]  # jiku1とjiku3 からの流しが無い...！

    return pairs

In [14]:
def buy_tickets(pairs, ratio, max_w):
    ws = 0
    boughts = []

    for idx in range(len(pairs)):
        if ws >= max_w:
            break

        bet = math.floor(float(max_w) * ratio[idx])
        ws += bet

        kumiban = ''
        if len(pairs[idx]) == 2:
            kumiban = "{0}-{1}".format(pairs[idx][0], pairs[idx][1])
        elif len(pairs[idx]) == 3:
            kumiban = "{0}-{1}-{2}".format(pairs[idx][0], pairs[idx][1], pairs[idx][2])
        else:
            kumiban = "{0}".format(pairs[idx][0])

        boughts.append({'kumiban': kumiban, 'bet': bet})

    idx = 0
    for j in range(max_w - ws):
        boughts[idx]['bet'] += 1
        idx += 1
        if idx >= len(boughts):
            idx = 0

    return boughts

## 枠番の追加

※ inken の取得したデータに枠番が入ってなかったので馬番から計算しました.

In [15]:
def uma2waku(umakazu, umaban):
    d = umaban2waku(int(umakazu))
    v = None
    try:
        v = d[int(umaban)]
    except:
        print(umakazu, umaban)
    return d[int(umaban)]

scores['waku'] = scores.apply(lambda x: uma2waku(x['umakazu'], x['umaban']), axis=1)
scores.head()

,race_info_id,uma_id,umaban,umakazu,score,value,waku
0,192991,14036,2,8,0.529314,1,2
1,192991,13703,3,8,0.306875,1,3
2,192991,12891,4,8,0.569858,1,4
3,192991,13319,5,8,0.370065,1,5
4,192991,59473,6,8,0.402599,1,6


## 式別毎購入可能点数の計算

予算範囲内で購入可能な点数を式別ごとに算出しておきます.

In [16]:
def count_marks(sub, select_num, uma_num):
    if sub == 'none':
        return select_num
    elif sub == 'box':
        return comb(select_num, uma_num, exact=True)
    elif sub == 'mbox':   # 単裏表
        return perm(select_num, uma_num, exact=True)
    elif sub == 'flow':   # 馬連などの流し
        return select_num - 1
    elif sub == 'mflow':  # 馬単マルチ
        return (select_num - 1) * 2
    elif sub == 'form2':  # 2頭フォーメーション
        return (select_num - 1) * 2 
    elif sub == 'flow1':  # 三連複軸1頭
        return comb(select_num - 1, 2, exact=True)
    elif sub == 'flow2':  # 三連複軸2頭
        return select_num - 2
    elif sub == 'form21':  # 三連複軸1頭-2頭フォーメーション
        return comb(select_num - 1, 2, exact=True) + comb(select_num - 2, 2, exact=True)
    elif sub == 'form22':  # 三連複軸2頭-2頭フォーメーション
        return (select_num - 2) + (select_num - 3)
    elif sub == 'tflow1':  # 三連単軸1頭固定
        return perm(select_num - 1, 2, exact=True)
    elif sub == 'tflow2':  # 三連単軸2頭固定
        return select_num - 2
    elif sub == 'mflow1':  # 三連単軸1頭マルチ
        return perm(select_num - 1, 2, exact=True) * 3
    elif sub == 'mflow2':  # 三連単軸2頭マルチ
        return (select_num - 2) * 6
    elif sub == 'form2t1': # 三連単軸1頭-2頭フォーメーション
        return perm(select_num - 1, 2, exact=True) + perm(select_num - 2, 2, exact=True)
    elif sub == 'form2t2':  # 三連単軸2頭-2頭フォーメーション
        return (select_num - 2) + (select_num - 3)

In [17]:
def search_threshold(sub, max_w, s, m):
    res = s
    for n in range(s, m + 1):
        mx = count_marks(sub, n, s)
        if mx > max_w:
            break
        res = n
    return res

In [18]:
MAX_PAYMENT = 2100  # 1レース当たりの購入上限を¥2,100 としています
MAX_UMAKAZU = 16
MAX_WAKUKAZU = 8
unit = 100    # 購入券の最小金額
max_w = int(MAX_PAYMENT / unit)

## 設定した金額で買える最大組数を探します
s_config = {}


# 単勝, 複勝
s_config['none'] = {
    '単勝': { 'low': 1, 'high': min([MAX_UMAKAZU, max_w]) },
    '複勝': { 'low': 1, 'high': min([MAX_UMAKAZU, max_w]) },
}

#------------------------------------------------------
# box
sub = 'box'
uma_num = 2

mx = search_threshold(sub, max_w, uma_num, MAX_UMAKAZU)
s_config[sub] = {
    '普通馬複': { 'low': uma_num, 'high': mx },
    'ワイド': { 'low': uma_num, 'high': mx },    
    '枠複': { 'low': uma_num, 'high': search_threshold(sub, max_w, uma_num, MAX_WAKUKAZU) },
    '三連複': { 'low': 3, 'high': search_threshold(sub, max_w, 3, MAX_UMAKAZU) },
}

#------------------------------------------------------
# mbox 単裏表
sub = 'mbox'
uma_num = 2

s_config[sub] = {
    '馬単': { 'low': uma_num, 'high': search_threshold(sub, max_w, uma_num, MAX_UMAKAZU) },
    '枠単': { 'low': uma_num, 'high': search_threshold(sub, max_w, uma_num, MAX_WAKUKAZU) },
    '三連単': { 'low': 3, 'high': search_threshold(sub, max_w, 3, MAX_UMAKAZU) },
}

#------------------------------------------------------
# flow 流し
sub = 'flow'
uma_num = 2

mx = min([search_threshold(sub, max_w, uma_num, MAX_UMAKAZU), MAX_UMAKAZU])
mxw = min([search_threshold(sub, max_w, uma_num, MAX_WAKUKAZU), MAX_WAKUKAZU])
s_config[sub] = {
    '普通馬複': { 'low': uma_num, 'high': mx },
    'ワイド': { 'low': uma_num, 'high': mx },
    '馬単': { 'low': uma_num, 'high': mx },
    '枠複': { 'low': uma_num, 'high': mxw },
    '枠単': { 'low': uma_num, 'high': mxw },
}

#------------------------------------------------------
# mflow 馬単マルチ
sub = 'mflow'
uma_num = 2

s_config[sub] = {
    '枠単': { 'low': uma_num, 'high': min([search_threshold(sub, max_w, uma_num, MAX_WAKUKAZU), MAX_WAKUKAZU]) },
    '馬単': { 'low': uma_num, 'high': min([search_threshold(sub, max_w, uma_num, MAX_UMAKAZU), MAX_UMAKAZU]) },
}

#------------------------------------------------------
# form2 フォーメーション
sub = 'form2'
uma_num = 2

mx = min([search_threshold(sub, max_w, uma_num, MAX_UMAKAZU), MAX_UMAKAZU])
mxw = min([search_threshold(sub, max_w, uma_num, MAX_WAKUKAZU), MAX_WAKUKAZU])
s_config[sub] = {
    '普通馬複': { 'low': 4, 'high': mx },
    'ワイド': { 'low': 4, 'high': mx },
    '枠複': { 'low': 4, 'high': mxw },
}


#------------------------------------------------------
# 三連複 flow1, flow2
uma_num = 3

sub = 'flow1'  # 三連複軸1頭
s_config[sub] = {
    '三連複': { 'low': uma_num, 'high': min([search_threshold(sub, max_w, uma_num, MAX_UMAKAZU), MAX_UMAKAZU]) },
}
sub = 'flow2'  # 三連複軸2頭
s_config[sub] = {
    '三連複': { 'low': uma_num, 'high': min([search_threshold(sub, max_w, uma_num, MAX_UMAKAZU), MAX_UMAKAZU]) },    
}

#------------------------------------------------------
# form21, form22 三連複フォーメーション
uma_num = 3

sub = 'form21'
s_config[sub] = {
    '三連複': { 'low': 4, 'high': min([search_threshold(sub, max_w, uma_num, MAX_UMAKAZU), MAX_UMAKAZU]) },
}

sub = 'form22'
s_config[sub] = {
    '三連複': { 'low': 4, 'high': min([search_threshold(sub, max_w, uma_num, MAX_UMAKAZU), MAX_UMAKAZU]) },
}

#------------------------------------------------------
# 三連単 tflow1, tflow2, mflow1, mflow2
uma_num = 3

sub = 'tflow1'  # 三連単軸1頭固定
s_config[sub] = {
    '三連単': { 'low': uma_num, 'high': min([search_threshold(sub, max_w, uma_num, MAX_UMAKAZU), MAX_UMAKAZU]) },
}
sub = 'tflow2'  # 三連単軸2頭固定
s_config[sub] = {
    '三連単': { 'low': uma_num, 'high': min([search_threshold(sub, max_w, uma_num, MAX_UMAKAZU), MAX_UMAKAZU]) },
}
sub = 'mflow1'  # 三連単軸1頭マルチ
s_config[sub] = {
    '三連単': { 'low': uma_num, 'high': min([search_threshold(sub, max_w, uma_num, MAX_UMAKAZU), MAX_UMAKAZU]) },
}
sub = 'mflow2'  # 三連単軸2頭マルチ
s_config[sub] = {
    '三連単': { 'low': uma_num, 'high': min([search_threshold(sub, max_w, uma_num, MAX_UMAKAZU), MAX_UMAKAZU]) },
}

#------------------------------------------------------
# form2t1, form2t2 三連単フォーメーション
uma_num = 3

sub = 'form2t1'
s_config[sub] = {
    '三連単': { 'low': 4, 'high': min([search_threshold(sub, max_w, uma_num, MAX_UMAKAZU), MAX_UMAKAZU]) },
}

sub = 'form2t2'
s_config[sub] = {
    '三連単': { 'low': 4, 'high': min([search_threshold(sub, max_w, uma_num, MAX_UMAKAZU), MAX_UMAKAZU]) },
}


In [19]:
types = {}
for k, v in s_config.items():
    for shiki, m in v.items():
        if shiki not in types:
            types[shiki] = {k: m}
        else:
            types[shiki][k] = m
types

{'単勝': {'none': {'low': 1, 'high': 16}},
 '複勝': {'none': {'low': 1, 'high': 16}},
 '普通馬複': {'box': {'low': 2, 'high': 7},
  'flow': {'low': 2, 'high': 16},
  'form2': {'low': 4, 'high': 11}},
 'ワイド': {'box': {'low': 2, 'high': 7},
  'flow': {'low': 2, 'high': 16},
  'form2': {'low': 4, 'high': 11}},
 '枠複': {'box': {'low': 2, 'high': 7},
  'flow': {'low': 2, 'high': 8},
  'form2': {'low': 4, 'high': 8}},
 '三連複': {'box': {'low': 3, 'high': 6},
  'flow1': {'low': 3, 'high': 8},
  'flow2': {'low': 3, 'high': 16},
  'form21': {'low': 4, 'high': 6},
  'form22': {'low': 4, 'high': 13}},
 '馬単': {'mbox': {'low': 2, 'high': 5},
  'flow': {'low': 2, 'high': 16},
  'mflow': {'low': 2, 'high': 11}},
 '枠単': {'mbox': {'low': 2, 'high': 5},
  'flow': {'low': 2, 'high': 8},
  'mflow': {'low': 2, 'high': 8}},
 '三連単': {'mbox': {'low': 3, 'high': 3},
  'tflow1': {'low': 3, 'high': 6},
  'tflow2': {'low': 3, 'high': 16},
  'mflow1': {'low': 3, 'high': 4},
  'mflow2': {'low': 3, 'high': 5},
  'form2t1': {'l

In [74]:
UMAKAZU = 16  # 5〜16

targets = df[df['umakazu'] == UMAKAZU]
targets.head()

,race_info_id,race_date,place,race_num,umakazu
155,193733,2019-01-21,大井,3,16
159,193737,2019-01-21,大井,7,16
160,193738,2019-01-21,大井,8,16
164,193742,2019-01-21,大井,12,16
166,193765,2019-01-22,大井,2,16


In [75]:
if UMAKAZU > 8:
    result_types = ['単勝', '複勝', '枠複', '枠単', '普通馬複', '馬単', 'ワイド', '三連複', '三連単']
else:
    result_types = ['単勝', '複勝', '普通馬複', '馬単', 'ワイド', '三連複', '三連単']

shiki_list = []
for k, v in types.items():
    for s in v.keys():
        shiki_list.append("{0}-{1}".format(k, s))
print("種類数", len(shiki_list))
shiki_list

種類数 29


['単勝-none',
 '複勝-none',
 '普通馬複-box',
 '普通馬複-flow',
 '普通馬複-form2',
 'ワイド-box',
 'ワイド-flow',
 'ワイド-form2',
 '枠複-box',
 '枠複-flow',
 '枠複-form2',
 '三連複-box',
 '三連複-flow1',
 '三連複-flow2',
 '三連複-form21',
 '三連複-form22',
 '馬単-mbox',
 '馬単-flow',
 '馬単-mflow',
 '枠単-mbox',
 '枠単-flow',
 '枠単-mflow',
 '三連単-mbox',
 '三連単-tflow1',
 '三連単-tflow2',
 '三連単-mflow1',
 '三連単-mflow2',
 '三連単-form2t1',
 '三連単-form2t2']

In [76]:
def objective(trial):

    targets = df[df['umakazu'] == UMAKAZU]

    # shiki: 式別 '単勝', '複勝', '枠複', '枠単', '普通馬複', '馬単', 'ワイド', '三連複', '三連単'
    shiki_sub = trial.suggest_categorical('shiki', shiki_list)

    # pattern: BOX, 流し, マルチなど
#     sub = trial.suggest_categorical('sub', list(types[shiki].keys()))  # カテゴリー値は動的変数にできなかった(´・ω・｀)ｼｮﾎﾞｰﾝ
    shiki, sub = shiki_sub.split('-')


    # ticket_unit: 式別の単位数
    if shiki in ['単勝', '複勝']:
        ticket_unit = 1
    elif shiki in ['三連複', '三連単']:
        ticket_unit = 3
    else:
        ticket_unit = 2


    # pattern: 均等買い or 重み買い
    if sub in ['none', 'flow', 'flow2', 'tflow2']:
        pattern = trial.suggest_categorical('pattern', ['eq', 'order_s', 'order_o'])
    else:
        pattern = 'eq'


    # select_num: 選択馬数
    select_num = trial.suggest_int('select_num', types[shiki][sub]['low'], min([types[shiki][sub]['high'], UMAKAZU]))


    print(shiki, sub, select_num, pattern, trial.number)

    # この値を最大化します
    profit = 0

    for i in list(targets['race_info_id']):
        ss = pd.DataFrame(scores[scores['race_info_id'] == i].sort_values('score', ascending=False))

        if shiki in ['枠複', '枠単']:
            ss['kumiban'] = ss['waku']
        else:
            ss['kumiban'] = ss['umaban']

        start_index = 0
        if sub in ['flow2', 'tflow2']:
            start_index = 2
        elif sub in ['flow']:
            start_index = 1

        rdf = pd.DataFrame(ss)
        rdf = rdf[start_index:select_num]
        rdf = rdf.reset_index()

        marks = count_marks(sub, select_num, ticket_unit)
        if pattern == 'eq':
            ratio = [ (1.0/float(marks)) for _ in range(marks)]
        elif pattern == 'order_s':
            ratio = list(rdf.loc[:, ['score']].apply(lambda x:x/sum(x))['score'])
        elif pattern == 'order_o':
            rdf['order'] = rdf.index * -1 + select_num
            ratio = list(rdf.loc[:, ['order']].apply(lambda x:x/sum(x))['order'])

        ss = ss[:select_num]
        ss = ss.reset_index()


        pairs = []
        if shiki == '単勝' or shiki == '複勝':
            pairs = tanpuku(ss, select_num)
        elif shiki == '普通馬複' or shiki == 'ワイド' or shiki == '枠複':
            pairs = uma_ren(ss, sub, select_num, ticket_unit)
        elif shiki == '馬単' or shiki == '枠単':
            pairs = uma_tan(ss, sub, select_num, ticket_unit)
        elif shiki == '三連複':
            pairs = sanren_puku(ss, sub, select_num, ticket_unit)
        elif shiki == '三連単':
            pairs = sanren_tan(ss, sub, select_num, ticket_unit)


        boughts = buy_tickets(pairs, ratio, max_w)


#         print(boughts)

        # 答え合わせ
        for b in boughts:
            res = results[(results['race_info_id'] == i) & (results['result_type'] == shiki) & (results['kumiban'] == b['kumiban'])]
            loss = int(list(res.price)[0]) if len(res) > 0 else 0.0
#             print(((loss - unit) * b['bet']), b['bet'], (loss - unit))
            profit = profit + ((loss - unit) * b['bet'])

    print("profit", profit)
    return -1.0 * profit

In [77]:
study = optuna.create_study()
study.optimize(objective, n_trials=300)

枠複 form2 5 eq 0
profit -80430.0


[I 2019-06-27 16:07:25,064] Finished trial#0 resulted in value: 80430.0. Current best value is 80430.0 with parameters: {'shiki': '枠複-form2', 'select_num': 5}.


単勝 none 13 eq 1
profit -87630.0


[I 2019-06-27 16:07:37,284] Finished trial#1 resulted in value: 87630.0. Current best value is 80430.0 with parameters: {'shiki': '枠複-form2', 'select_num': 5}.


三連単 mflow2 5 eq 2
profit -90900.0


...

[I 2019-06-27 17:03:56,877] Finished trial#297 resulted in value: 66520.0. Current best value is -108830.0 with parameters: {'shiki': '三連単-tflow2', 'pattern': 'order_s', 'select_num': 11}.


枠単 flow 6 order_s 298
profit -55780.0


[I 2019-06-27 17:04:02,698] Finished trial#298 resulted in value: 55780.0. Current best value is -108830.0 with parameters: {'shiki': '三連単-tflow2', 'pattern': 'order_s', 'select_num': 11}.


三連複 box 5 eq 299
profit -80280.0


[I 2019-06-27 17:04:12,303] Finished trial#299 resulted in value: 80280.0. Current best value is -108830.0 with parameters: {'shiki': '三連単-tflow2', 'pattern': 'order_s', 'select_num': 11}.


In [79]:
study.best_params

{'shiki': '三連単-tflow2', 'pattern': 'order_s', 'select_num': 11}

In [80]:
UMAKAZU

16

# 結果

### 6頭

```
{'shiki': '三連単-tflow2', 'pattern': 'eq', 'select_num': 4}
```
対象レース: 6

### 7頭

+190円

```
{'shiki': '馬単-mbox', 'select_num': 2}
```
対象レース: 16

### 8頭

-300円

```
{'shiki': '単勝-none', 'pattern': 'eq', 'select_num': 7}
```
対象レース: 32

### 9頭

+11,970円

```
{'shiki': '三連複-flow1', 'select_num': 3}
```
対象レース: 69

### 10頭

+35,980円

```
{'shiki': '三連単-form2t2', 'select_num': 10}
```
対象レース: 99

### 11頭

+307,560円

```
{'shiki': '三連単-mbox', 'select_num': 3}
```

対象レース: 198

### 12頭

+26,910円

```
{'shiki': '三連単-form2t2', 'select_num': 8}
```

対象レース: 503

### 13頭

+170,910円

```
{'shiki': '馬単-flow', 'pattern': 'eq', 'select_num': 11}
```

対象レース: 85

### 14頭

+218,930円

```
{'shiki': '三連複-form22', 'select_num': 5}
```

対象レース: 202

### 15頭

+53,240円

```
{'shiki': '馬単-flow', 'pattern': 'eq', 'select_num': 11}
```
対象レース: 70


### 16頭

+108,830円

```
{'shiki': '三連単-tflow2', 'pattern': 'order_s', 'select_num': 11}
```
対象レース: 147
